In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel


ModuleNotFoundError: No module named 'pandas'

In [1]:
df = pd.read_excel("UX-Daten.xlsx", sheet_name="Formularantworten 1")
df.head()


NameError: name 'pd' is not defined

In [ ]:
df_long = df.melt(
    id_vars=["TeilnehmerID"],
    value_vars=[col for col in df.columns if col.startswith("AR ") or col.startswith("Bild ")],
    var_name="ItemBedingung",
    value_name="Wert"
)

df_long["Bedingung"] = df_long["ItemBedingung"].apply(lambda x: "AR" if x.startswith("AR ") else "Bild")
df_long["Item"] = df_long["ItemBedingung"].str.replace("^(AR |Bild )", "", regex=True)
df_long = df_long[["TeilnehmerID", "Item", "Bedingung", "Wert"]]
df_long.head()


In [ ]:
pivot = df_long.pivot(index=["TeilnehmerID", "Item"], columns="Bedingung", values="Wert").dropna()

results = []
for item in pivot.index.get_level_values("Item").unique():
    ar = pivot.xs(item, level="Item")["AR"]
    bild = pivot.xs(item, level="Item")["Bild"]
    t, p = ttest_rel(ar, bild)
    d = (ar.mean() - bild.mean()) / np.sqrt(((ar.std(ddof=1)**2 + bild.std(ddof=1)**2)/2))
    results.append({"Item": item, "t-Wert": t, "p-Wert": p, "Cohen's d": d})

df_results = pd.DataFrame(results)
df_results


In [ ]:
df_results.to_excel("auswertung_ergebnisse.xlsx", index=False)
